In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader

directorio = f"./DATA/asistente_2"
pdf_files = [f for f in os.listdir(directorio) if f.endswith('.pdf')]
documentos = []
for pdf_file in pdf_files:
    ruta_pdf = os.path.join(directorio, pdf_file)
    loader = PyPDFLoader(ruta_pdf)
    documentos.extend(loader.load())



In [4]:
len(documentos)

11

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len,
        add_start_index=True,
    )

chunks = text_splitter.split_documents(documentos)
len(chunks)

62

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import shutil

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

CHROMA_PATH = "CHROMA/asistente_2"

if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

db = Chroma.from_documents(
        chunks, embedding_function, persist_directory=CHROMA_PATH
    )

db.persist()


C:\Users\adm_hlopez\AppData\Local\Temp\ipykernel_10408\1394854328.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [9]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

C:\Users\adm_hlopez\AppData\Local\Temp\ipykernel_10408\244422769.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)


In [39]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

query_text="que se entiende por uso aceptable"

results = db.similarity_search_with_relevance_scores(query_text, k=3)
#print(results)

PROMPT_TEMPLATE = """
Responde la pregunta basandote solo en el siguiente contexto:

{context}

---

Responde la pregunta basandote en el contexto de arriba: {question}
"""

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
#print(prompt)
model = ChatOllama(model="mistral",temperature=0,stream=True)
response_text = model.predict(prompt)
print(response_text)

31. El uso aceptable, según el contexto proporcionado, se refiere a la utilización de manera autorizada y en estricta conformidad con las políticas y normativas de seguridad de la institución, con el fin de preservar la integridad, confidencialidad y disponibilidad de la información.

32. El propietario del activo de información corresponde al responsable de tomar decisiones respecto del activo. Esto no implica necesariamente derecho de propiedad sobre el activo de información.

---

• Definir los procesos de gestión y cambios correspondiente a los activos de información se refiere a establecer procedimientos para administrar y modificar los activos de información críticos en la Subsecretaría, según sea indicado por las jefaturas de cada área.
• Definir el uso aceptable del activo de la información tanto como a los funcionarios/as como a las partes interesadas se refiere a establecer normas y reglas para la utilización autorizada de los activos de información críticos por parte de los 

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rephrase_template = """Dada la siguiente conversación y una pregunta de seguimiento, reformule la pregunta de seguimiento para que sea una pregunta independiente, en su idioma original..

Historial de chat:
{chat_history}
Entrada de seguimiento: {question}
Pregunta independiente:"""

REPHRASE_TEMPLATE = PromptTemplate.from_template(rephrase_template)
rephrase_chain = REPHRASE_TEMPLATE | ChatOllama(model="mistral",temperature=0) | StrOutputParser()

template = """Como un asistente virtual del Ministerio de energia, 
Responda la pregunta  basándose únicamente en el siguiente contexto.:

{context}

Pregunta: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | ANSWER_PROMPT
    | ChatOllama(model="mistral",temperature=0)
    | StrOutputParser()
)

final_chain = rephrase_chain | retrieval_chain

In [47]:
final_chain.invoke(
    {
        "question": "cual es la capital de francia",
        "chat_history": [
           
        ],
    }
)

'¡Lo siento! El contexto proporcionado no contiene información sobre la capital de Francia.'